In [19]:
import os
import time
import pandas as pd
import pdfplumber
from openai import OpenAI
from typing import List, Dict, Any
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor

In [20]:

# Configuración inicial
PDF_PATH = r"C:\Users\maria\Desktop\data\we are social\Digital_2025_Spain_v02.pdf"  # Ruta al archivo PDF
OUTPUT_EXCEL = "datos_estructurados.xlsx"  # Archivo Excel de salida
OPENAI_API_KEY = "tu_api_key_aqui"  # Tu API key de OpenAI
MAX_WORKERS = 4  # Número de trabajadores concurrentes (ajustar según necesidad)
# Inicializar cliente de OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

In [21]:
def extract_text_from_page(pdf_path: str, page_num: int) -> str:
    """Extrae el texto de una página específica del PDF"""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            if page_num < len(pdf.pages):
                page = pdf.pages[page_num]
                return page.extract_text() or ""
            else:
                return ""
    except Exception as e:
        print(f"Error al extraer texto de la página {page_num}: {e}")
        return ""

In [22]:
def extract_text_from_page(pdf_path: str, page_num: int) -> str:
    """Extrae el texto de una página específica del PDF"""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            if page_num < len(pdf.pages):
                page = pdf.pages[page_num]
                return page.extract_text() or ""
            else:
                return ""
    except Exception as e:
        print(f"Error al extraer texto de la página {page_num}: {e}")
        return ""

In [23]:
def translate_and_extract_data(text: str, page_num: int) -> Dict[str, Any]:
    """Traduce el texto al español y extrae datos estructurados usando IA"""
    if not text.strip():
        return {"page_number": page_num, "data": {}, "error": "Página vacía o sin texto"}
    
    try:
        # Prompt para la IA que realiza la traducción y extracción
        prompt = f"""
        Esta es la página {page_num} de un informe. Por favor:
        
        1. Traduce el siguiente texto del inglés al español:
        
        {text}
        
        2. Después de traducir, identifica y extrae todos los datos numéricos, estadísticas, 
        porcentajes y métricas relevantes que aparezcan en el texto. Devuelve estos datos 
        en formato JSON, donde cada clave sea el nombre del dato/métrica (en español) y 
        el valor sea el dato numérico con su unidad.
        
        3. Si hay tablas o gráficos descritos, extrae también esa información estructurada.
        
        Por favor, proporciona dos secciones:
        1. "traduccion": El texto completo traducido
        2. "datos": Un objeto JSON con los datos estructurados extraídos
        """
        
        # Llamada a la API de OpenAI
        response = client.chat.completions.create(
            model="gpt-4o",  # Puedes usar gpt-3.5-turbo para reducir costos
            messages=[
                {"role": "system", "content": "Eres un asistente experto en traducción y extracción de datos de informes estadísticos."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1,  # Temperatura baja para respuestas más consistentes
            response_format={"type": "json_object"}
        )
        
        # Procesar la respuesta y extraer datos
        result = response.choices[0].message.content
        
        # Convertir la respuesta a formato JSON (la respuesta ya debe venir en formato JSON)
        import json
        parsed_result = json.loads(result)
        
        return {
            "page_number": page_num,
            "translation": parsed_result.get("traduccion", ""),
            "data": parsed_result.get("datos", {}),
            "error": None
        }
        
    except Exception as e:
        print(f"Error en la página {page_num}: {e}")
        return {
            "page_number": page_num,
            "translation": "",
            "data": {},
            "error": str(e)}

In [24]:
def process_pdf(pdf_path: str, start_page: int = 0, end_page: int = None) -> List[Dict[str, Any]]:
    """Procesa el PDF completo o un rango de páginas especificado"""
    results = []
    
    # Determinar el número total de páginas
    with pdfplumber.open(pdf_path) as pdf:
        total_pages = len(pdf.pages)
    
    if end_page is None or end_page > total_pages:
        end_page = total_pages
    
    print(f"Procesando PDF: {pdf_path}")
    print(f"Total de páginas a procesar: {end_page - start_page}")
    
    # Lista para almacenar las tareas
    tasks = []
    
    # Crear tareas para el procesamiento concurrente
    for page_num in range(start_page, end_page):
        # Extraer texto de la página
        text = extract_text_from_page(pdf_path, page_num)
        tasks.append((text, page_num))
    
    # Procesar páginas con múltiples hilos
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Mapear cada tarea al método translate_and_extract_data
        future_to_page = {executor.submit(translate_and_extract_data, text, page_num): page_num 
                          for text, page_num in tasks}
        
        # Recopilar resultados a medida que se completan
        for i, future in enumerate(concurrent.futures.as_completed(future_to_page)):
            page_num = future_to_page[future]
            try:
                result = future.result()
                results.append(result)
                
                # Imprimir progreso
                print(f"Completado {i+1}/{len(tasks)}: Página {page_num+1}")
                
                # Pausa breve para no sobrecargar la API
                time.sleep(0.5)
                
            except Exception as e:
                print(f"Error procesando página {page_num+1}: {e}")
                results.append({
                    "page_number": page_num,
                    "translation": "",
                    "data": {},
                    "error": str(e)
                })
    
    # Ordenar resultados por número de página
    results.sort(key=lambda x: x["page_number"])
    return results

In [25]:

def save_to_excel(results: List[Dict[str, Any]], output_file: str):
    """Guarda los resultados en un archivo Excel"""
    # Crear un DataFrame para la hoja de datos estructurados
    all_data = []
    
    for result in results:
        page_num = result["page_number"] + 1  # +1 para que empiece en 1 en lugar de 0
        data = result["data"]
        
        # Agregar cada dato con su página de origen
        for key, value in data.items():
            all_data.append({
                "Página": page_num,
                "Métrica": key,
                "Valor": value
            })
    
    # Crear DataFrame
    df_data = pd.DataFrame(all_data)
    
    # Crear un Excel con múltiples hojas
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Hoja 1: Datos estructurados
        if not df_data.empty:
            df_data.to_excel(writer, sheet_name='Datos Estructurados', index=False)
        
        # Hoja 2: Traducciones completas
        translations = [{
            "Página": result["page_number"] + 1,
            "Texto Traducido": result["translation"]
        } for result in results]
        
        df_translations = pd.DataFrame(translations)
        df_translations.to_excel(writer, sheet_name='Traducciones', index=False)
        
        # Hoja 3: Registro de errores
        errors = [{
            "Página": result["page_number"] + 1,
            "Error": result["error"]
        } for result in results if result["error"]]
        
        if errors:
            df_errors = pd.DataFrame(errors)
            df_errors.to_excel(writer, sheet_name='Errores', index=False)
    
    print(f"Resultados guardados en: {output_file}")



In [26]:
def main():
    """Función principal"""
    start_time = time.time()
    
    # Procesar el PDF completo
    results = process_pdf(PDF_PATH)
    
    # Guardar resultados en Excel
    save_to_excel(results, OUTPUT_EXCEL)
    
    # Calcular tiempo total de procesamiento
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Proceso completado en {elapsed_time / 60:.2f} minutos")

if __name__ == "__main__":
    main()

Procesando PDF: C:\Users\maria\Desktop\data\we are social\Digital_2025_Spain_v02.pdf
Total de páginas a procesar: 167
Completado 1/167: Página 2
Error en la página 0: Error code: 401 - {'error': {'message': 'Incorrect API key provided: tu_api_k***aqui. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Completado 2/167: Página 1
Error en la página 4: Error code: 401 - {'error': {'message': 'Incorrect API key provided: tu_api_k***aqui. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error en la página 2: Error code: 401 - {'error': {'message': 'Incorrect API key provided: tu_api_k***aqui. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error en la página 3: Error code: 401 - {'err

IndexError: At least one sheet must be visible